In [76]:
import requests
import json
from dotenv import load_dotenv
import os
load_dotenv()  # have your rpc endpoint in .env file to prevent accidental commit to github

to_address = '0x50f9202e0f1c1577822bd67193960b213cd2f331' # hacker EOA
contract_address = '9eb5f8e83359bb5013f3d8eee60bdce5654e8881' # hacker contract
approve_method_id = '095ea7b3'
url = "https://q.luabase.com/run"

def queryLuabase(sql):
    headers = {"content-type": "application/json"}
    payload = {
        "block": {
            "details": {
            "sql": sql,
            }
        },
        "api_key": os.environ.get('LUABASE_API_KEY')
    }

    res = requests.request("POST", url, json=payload, headers=headers).json()
    if res['data'] is not None:
        return res['data']
    return res

In [36]:
# collect list of erc20 tranfer txs that were sent to the hacker's address
transfer_data = queryLuabase(f"select from_address as victim_from_address,token_address as erc20_token_address,to_address as hacker_to_address,transaction_hash, value as amount FROM ethereum.token_transfers AS t WHERE t.to_address = '{to_address}' AND t.block_number >= 15213261")
len(transfer_data)


11

In [37]:
# collect list of txs that have the withdraw erc20 balance method id of victim's addresses & the address called is the hacker's contract. 
transaction_data = queryLuabase(f"select from_address,to_address,hash,block_number FROM ethereum.transactions AS t where t.input LIKE '%0x9c307de6%' AND t.to_address = '0x{contract_address}' AND t.block_number >= 15213261")
len(transaction_data)

9

In [38]:
# collect victim's addresses
final_data = []
for t in transfer_data:
    for tx in transaction_data:
        if t['transaction_hash'] == tx['hash']:
            # print(tx)
            t['hacker_contract_address'] = tx['to_address']
            final_data.append(t)

print(len(final_data))
final_data[0]

9


{'victim_from_address': '0x1a2a910185467dab98c3c04eee38ac6dbbc3b0d5',
 'erc20_token_address': '0x6b175474e89094c44da98b954eedeac495271d0f',
 'hacker_to_address': '0x50f9202e0f1c1577822bd67193960b213cd2f331',
 'transaction_hash': '0xc8e035ac6abd1d7a34a73ed66b7ea15a55d34db762bc073e0a29761975e6c448',
 'amount': 4000917429820000000000,
 'hacker_contract_address': '0x9eb5f8e83359bb5013f3d8eee60bdce5654e8881'}

In [39]:
# get erc20 information to calcuate the total assets stolen. 
unique_erc20s = {}

assets = []
for f in final_data:
    erc20 = f['erc20_token_address']
    if erc20 not in unique_erc20s:
        response = queryLuabase(f"select symbol, decimals from ethereum.tokens as t WHERE t.address = '{erc20}'")[0]
        unique_erc20s[erc20] = {'erc20_decimals': response['decimals'], 'erc20_symbol': response['symbol']}
    f.update(unique_erc20s[erc20])

final_data[0]

{'victim_from_address': '0x1a2a910185467dab98c3c04eee38ac6dbbc3b0d5',
 'erc20_token_address': '0x6b175474e89094c44da98b954eedeac495271d0f',
 'hacker_to_address': '0x50f9202e0f1c1577822bd67193960b213cd2f331',
 'transaction_hash': '0xc8e035ac6abd1d7a34a73ed66b7ea15a55d34db762bc073e0a29761975e6c448',
 'amount': 4000917429820000000000,
 'hacker_contract_address': '0x9eb5f8e83359bb5013f3d8eee60bdce5654e8881',
 'erc20_decimals': 18,
 'erc20_symbol': 'DAI'}

In [40]:
# calculate total assets stolen based on the ERC20 information & transaction data retrieved

asset_lost_results = {}

for f in final_data:
    erc20 = f['erc20_token_address']
    if erc20 not in asset_lost_results:
        asset_lost_results[erc20] = {'amount': f['amount'], 'decimals': f['erc20_decimals'], 'symbol': f['erc20_symbol']}
    else:
        asset_lost_results[erc20].update({'amount': f['amount'] + asset_lost_results[erc20]['amount']})

for asset in asset_lost_results.keys():
    asset_lost_results[asset]['amount'] = asset_lost_results[asset]['amount'] / pow(10, asset_lost_results[asset]['decimals'])

asset_lost_results

{'0x6b175474e89094c44da98b954eedeac495271d0f': {'amount': 6458.823763845549,
  'decimals': 18,
  'symbol': 'DAI'},
 '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48': {'amount': 606265.325268,
  'decimals': 6,
  'symbol': 'USDC'}}

In [41]:
# get all approved transactions of the victim's addresses.

approved_txs = {}

for asset in asset_lost_results.keys():
    response = queryLuabase(f"select hash,block_number, input,from_address from ethereum.transactions WHERE to_address = '{asset}' AND input LIKE '%0x{approve_method_id}000000000000000000000000{contract_address}%' AND block_number < 15310286")
    approved_txs[asset] = response

approved_txs


{'0x6b175474e89094c44da98b954eedeac495271d0f': [{'hash': '0x58fad2f53183fec4226e10bca0b3c96d3dff41e2a645f458bf2237ecf027e10d',
   'block_number': 15309954,
   'input': '0x095ea7b30000000000000000000000009eb5f8e83359bb5013f3d8eee60bdce5654e8881ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff',
   'from_address': '0x056fc75753bd75ea28b5966c4bea29f4ad49321c'},
  {'hash': '0x2acc9aa3cf6e47ec78050f49fef2aa309f37c0b1e5e7d68f16b4ae8465c2cd03',
   'block_number': 15310030,
   'input': '0x095ea7b30000000000000000000000009eb5f8e83359bb5013f3d8eee60bdce5654e8881ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff',
   'from_address': '0xb3e08599ac57666be68dbb3d311b9c607900a83b'},
  {'hash': '0x518e44788c095cec696291ef23e9c6b51914fc26186a047621663eb498993360',
   'block_number': 15310180,
   'input': '0x095ea7b30000000000000000000000009eb5f8e83359bb5013f3d8eee60bdce5654e88810000000000000000000000000000000000000000000000000000000000000000',
   'from_address': '0x1a2a9101

In [42]:
# get addresses that potentially did not get assets stolen

potential_victim_addresses = []

for key in approved_txs.keys():
    approved_txs[key] = list(filter(lambda k: k['input'] != '0x095ea7b30000000000000000000000009eb5f8e83359bb5013f3d8eee60bdce5654e88810000000000000000000000000000000000000000000000000000000000000000',approved_txs[key]))

    potential_victim_addresses.extend([ sub['from_address'] for sub in approved_txs[key] ])



victim_addresses = [ sub['victim_from_address'] for sub in final_data ]

# print(victim_addresses, len(victim_addresses))
# print(potential_victim_addresses, len(potential_victim_addresses))

approved_but_unaffected_victims = list(set(victim_addresses).symmetric_difference(set(potential_victim_addresses)))

print(approved_but_unaffected_victims)

['0xc705faf7a01faaf50198ae32a8e1e8b3a60b18bb', '0xb3e08599ac57666be68dbb3d311b9c607900a83b']


In [66]:
# query list of txs that hacker cashes out (eth to other platform). value_eth >= 0.1 based on etherscan
response = queryLuabase(f"select block_number, hash, to_address, value_eth, tag from ethereum.transactions as t left join ethereum.tags as tags on t.to_address = tags.address WHERE t.value_eth >= 0.1 and t.from_address = '{to_address}'")
response

            

[{'block_number': 15310351,
  'hash': '0xf2d0a2a2d8c9c178cb657724658b61f52b9c1028fd76ed945936493667ea4372',
  'to_address': '0xd90e2f925da726b50c4ed8d0fb90ad053324f31b',
  'value_eth': 1.0,
  'tag': 'Tornado.Cash: Router'},
 {'block_number': 15309715,
  'hash': '0xeaebd96cd99db2f27165d761f6d69c0222714214678c47541827d63a6a0f5359',
  'to_address': '0x45470961fb25789654fcfc391baf77019051ef78',
  'value_eth': 0.1,
  'tag': ''},
 {'block_number': 15310385,
  'hash': '0xf6e13e7c8606f181e6a61da62f523e771ed20d523356ce691a0ecfa0058dc379',
  'to_address': '0xd90e2f925da726b50c4ed8d0fb90ad053324f31b',
  'value_eth': 0.1,
  'tag': 'Tornado.Cash: Router'},
 {'block_number': 15310366,
  'hash': '0xd917bc4ae6207e0aad066eae5fd309c077b94aef2b1953c659f6faec2aa340cf',
  'to_address': '0xd90e2f925da726b50c4ed8d0fb90ad053324f31b',
  'value_eth': 1.0,
  'tag': 'Tornado.Cash: Router'},
 {'block_number': 15310377,
  'hash': '0xd118f1db41f3f46835d223511406c340b41f1f50cf741e0b1ec9945d68db5a64',
  'to_address': 

In [70]:
# filter cash out based on tag

cash_out_only_tags = []
cash_out_known_tags = []
cash_out_unknown_tags = []

for r in response:
    reduced_r = {'value_eth':r['value_eth'], 'tag': r['tag']}
    if r['tag'] == '':
        cash_out_unknown_tags.append(r)
        continue

    if r['tag'] not in cash_out_only_tags:
        cash_out_only_tags.append(r['tag'])
        cash_out_known_tags.append(reduced_r)
        continue

    for c in cash_out_known_tags:
        if r['tag'] == c['tag']:
            c['value_eth'] += r['value_eth']
            break

print(cash_out_known_tags)
print(cash_out_unknown_tags)

[{'value_eth': 27.700000000000003, 'tag': 'Tornado.Cash: Router'}]
[{'block_number': 15309715, 'hash': '0xeaebd96cd99db2f27165d761f6d69c0222714214678c47541827d63a6a0f5359', 'to_address': '0x45470961fb25789654fcfc391baf77019051ef78', 'value_eth': 0.1, 'tag': ''}, {'block_number': 15310223, 'hash': '0x2e76259606c3f4fc2842e1dce99c2631753c4526b7c615312f2a65b285689d48', 'to_address': '0x06c339416d76fce9ca8a7a66b4aedbe7301826d7', 'value_eth': 45.0, 'tag': ''}, {'block_number': 15310242, 'hash': '0x5108cd31c7a16727275d2f55fe2a31f2f3f20cd3821c485b1917d9411c97fb70', 'to_address': '0xc3e20d299084b7017ad44e6cd2ba0a1d39ab2e31', 'value_eth': 45.0, 'tag': ''}, {'block_number': 15310252, 'hash': '0x069620d1f2b81edfa8f6ff25e3f45f53e43378e7fc9f637f7d227ed8e27fbec8', 'to_address': '0xc137c19946724f76e65bb0efb81ddcf895d6472c', 'value_eth': 45.0, 'tag': ''}, {'block_number': 15310290, 'hash': '0x57da7f38aa453f08804de988bb093c6e359a90c3e753609f9d29e26d78225e30', 'to_address': '0xb6d318c4d1618eb8dcc2e8c0960

In [94]:
# collect cash out txs for unknown tags. These addresses are also from hacker's
# reuse var
cash_out_new_known_tags = []
for c in cash_out_unknown_tags:
    response = queryLuabase(f"select block_number, hash, to_address, value_eth, tag from ethereum.transactions as t left join ethereum.tags as tags on t.to_address = tags.address WHERE t.value_eth >= 0.1 and t.from_address = '{c['to_address']}'")
    cash_out_new_known_tags.append(response)

cash_out_new_known_tags

[[],
 [{'block_number': 15310245,
   'hash': '0x5cbe818b9bbfa4344e48c8bb5d37c0eecdcde8b15613c6823ff13d0af72c353d',
   'to_address': '0x4e5b2e1dc63f6b91cb6cd759936495434c7e972f',
   'value_eth': 44.99972700000001,
   'tag': 'FixedFloat'}],
 [{'block_number': 15310300,
   'hash': '0x22df8dee167bbf32bfecca2aa1aa33487402a3c840383f0b9b4dced215553d54',
   'to_address': '0x4e5b2e1dc63f6b91cb6cd759936495434c7e972f',
   'value_eth': 44.999706,
   'tag': 'FixedFloat'}],
 [{'block_number': 15310274,
   'hash': '0x64fdd3998e60b98058d524dd5c9ea7c22db039a276e3594501fd18aee9f08bd4',
   'to_address': '0x4e5b2e1dc63f6b91cb6cd759936495434c7e972f',
   'value_eth': 44.999391,
   'tag': 'FixedFloat'}],
 [{'block_number': 15310304,
   'hash': '0x81df74a39265d6f85e19ec0954c90e393d6df4bf7f250ddf0c31dfe580d1dbb1',
   'to_address': '0x4e5b2e1dc63f6b91cb6cd759936495434c7e972f',
   'value_eth': 21.999643,
   'tag': 'FixedFloat'}],
 [{'block_number': 15310229,
   'hash': '0x2eef64d081a222a9d1667d27caf76a1034eba96c

In [97]:
tags = []
only_tags = []

for c in cash_out_new_known_tags:
    for cc in c:
        reduced_c = {'value_eth':cc['value_eth'], 'tag': cc['tag']}
        if cc['tag'] == '':
            continue

        if cc['tag'] not in only_tags:
            only_tags.append(cc['tag'])
            tags.append(reduced_c)
            continue

        for t in tags:
            if cc['tag'] == t['tag']:
                t['value_eth'] += cc['value_eth']
                break

tags

[{'value_eth': 291.99741700000004, 'tag': 'FixedFloat'},
 {'value_eth': 19.997937, 'tag': 'Binance 14'}]